In [ ]:
import scipy.io
import numpy as np
import pandas as pd
import scanpy as sc
import h5py

import os
import numpy as np
import pandas as pd
import scanpy as sc
import anndata 
import seaborn as sns
from scipy.stats import zscore
import matplotlib.pyplot as plt
import collections
from natsort import natsorted

from scipy import stats
from scipy import sparse
from sklearn.decomposition import PCA
from umap import UMAP

from matplotlib.colors import LinearSegmentedColormap
from statsmodels.stats.multitest import multipletests

from scroutines.config_plots import *
from scroutines import powerplots # .config_plots import *
from scroutines import pnmf
from scroutines import basicu
from scroutines.gene_modules import GeneModules  

In [ ]:
%%time
adatas_pk = []
ts = [6,8,10,12,14,17,21]
for ti in ts:
    print(ti)
    f = f'/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/P{ti}_ATAC_L23_peakbycell_filtered.h5ad'
    adata_pk = sc.read(f)
    adata_pk.obs = adata_pk.obs.set_index('cell')
    print(adata_pk.var['peak'][:3].values)
    
    adatas_pk.append(adata_pk)
    print(adata_pk)
    break

allpeaks = adata_pk.var['peak'].values
adata_pk.var, allpeaks

In [ ]:
%%time
fout0 = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/samesign_b_vs_ac_p6to21.txt'
fout1 = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/qs_b_vs_ac_p6to21.txt'
fout2 = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/l2fc_b_vs_ac_p6to21.txt'

same_sign_alltime = np.loadtxt(fout0)
qs_alltime   = np.loadtxt(fout1)
l2fc_alltime = np.loadtxt(fout2)

In [ ]:
same_sign_alltime = np.array(same_sign_alltime)
qs_alltime = np.array(qs_alltime)
l2fc_alltime = np.array(l2fc_alltime)

In [ ]:
l2fc_th = np.log2(1.5)
alpha_th = 0.05
cond_sig    = np.all([same_sign_alltime > 0, qs_alltime < alpha_th, np.abs(l2fc_alltime) > l2fc_th], axis=0)
cond_sig_bp = np.all([same_sign_alltime > 0, qs_alltime < alpha_th, l2fc_alltime >  l2fc_th], axis=0)
cond_sig_bn = np.all([same_sign_alltime > 0, qs_alltime < alpha_th, l2fc_alltime < -l2fc_th], axis=0)

print('num B vs (A/C) DARs per cond:\t', cond_sig.sum(axis=1))
print('num B   per cond:\t',    cond_sig_bp.sum(axis=1))
print('num A/C per cond:\t',    cond_sig_bn.sum(axis=1))

In [ ]:
gene_sig_instances    = cond_sig.sum(axis=0)
gene_sig_instances_bp = cond_sig_bp.sum(axis=0)
gene_sig_instances_bn = cond_sig_bn.sum(axis=0)

instances, counts     = np.unique(gene_sig_instances, return_counts=True)
instances_bp, counts_bp = np.unique(gene_sig_instances_bp, return_counts=True)
instances_bn, counts_bn = np.unique(gene_sig_instances_bn, return_counts=True)

print('num B vs (A/C) DARs in num conds:\t',  counts   , np.any(cond_sig, axis=0).sum())
print('num B   DARs in num conds:\t',         counts_bp, np.any(cond_sig_bp, axis=0).sum())
print('num A/C DARs in num conds:\t',         counts_bn, np.any(cond_sig_bn, axis=0).sum())

In [ ]:
def split_peakstring(peak):
    """
    """
    a, bc = peak.split(':')
    b, c = bc.split('-')
    b, c = int(b), int(c)
    return a,b,c

def peaksstrings_to_bed(peaks, name=None, fout=None, **kwargs):
    """
    """
    
    bedarray = np.char.replace(np.char.replace(peaks, ':', '\t'), '-', '\t')
    if name is not None:
        lastcol = np.arange(len(peaks)).astype(str)
        linker = f"\t{name}_"
        bedarray = np.char.add(np.char.add(bedarray, linker), lastcol) 
        
    if fout is not None:
        print(f"saving to {fout}")
        np.savetxt(fout, bedarray, **kwargs)
        
    return bedarray

# [(a,b,c) for a,b,c in map(split_peakstring, a_peaks[:10])]
# peaksstrings_to_bed(a_peaks, name='P6_A')


In [ ]:
import subprocess
def get_closest_genes(peaks, dout, name):
    """run bedtools using subprocess
    """
    fout_peaks = dout + f'/{name}_peaks.bed'
    fout_cgene = dout + f'/{name}_peaks_ngene.bed'
    
    # save to bed
    peaksstrings_to_bed(peaks, name=name, fout=fout_peaks, fmt='%s')

    # run bedtools and save
    # !bedtools closest -g mm10.sorted.chrom.sizes -a test.bed -b genebody.bed > out.bed
    with open(fout_cgene, 'w') as f:
        print(f"Saving to {fout_cgene}")
        result = subprocess.run(["bedtools", "closest", 
                                    "-g", "mm10.sorted.chrom.sizes", 
                                    "-t", "first",
                                    "-a", fout_peaks,
                                    "-b", "gencode.vM25.genebody_pconly.bed", #"gencode.vM25.genebody_pconly.bed",
                                   ], cwd=dout, stdout=f, stderr=subprocess.PIPE, text=True)
    print(f"Return code: {result.returncode}")

    # read the results back
    df = pd.read_csv(fout_cgene, sep='\t', header=None)
    num_uniq_genes = len(df[7].unique())
    print(f"(pks; clst 1st g, uniq g): ({len(peaks)}, {len(df)}, {num_uniq_genes})")
    
    return df

In [ ]:
dout = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/results_atac'
!mkdir -p $dout

In [ ]:
%%time
for i in range(7):
    t = ts[i]
    print(i, t)
    peaks_bp = np.array(natsorted(allpeaks[cond_sig_bp[i]]))
    peaks_bn = np.array(natsorted(allpeaks[cond_sig_bn[i]]))
    
    if len(peaks_bp) > 0:
        res_bp = get_closest_genes(peaks_bp, dout, f'P{t}_Bp') 
    if len(peaks_bn) > 0:
        res_bn = get_closest_genes(peaks_bn, dout, f'P{t}_Bn') 